<a href="https://colab.research.google.com/github/brrikcy/nlp/blob/main/Word_embedding_using_CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import nltk
from nltk.corpus import movie_reviews
from nltk.tokenize import word_tokenize
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Conv1D, MaxPooling1D, Flatten, Dense

In [ ]:
# Download necessary NLTK data
nltk.download('movie_reviews')
nltk.download('punkt')

[nltk_data] Downloading package movie_reviews to /root/nltk_data...
[nltk_data]   Unzipping corpora/movie_reviews.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
# Load and prepare the dataset
documents = [(list(movie_reviews.words(fileid)), category)
             for category in movie_reviews.categories()
             for fileid in movie_reviews.fileids(category)]

texts = [' '.join(doc) for doc, _ in documents]
labels = [label for _, label in documents]

In [ ]:
# Encode labels
label_encoder = LabelEncoder()
labels_encoded = label_encoder.fit_transform(labels)

In [ ]:
# Split data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(texts, labels_encoded, test_size=0.2, random_state=42)

In [ ]:
# Tokenize and pad sequences
max_words = 10000  # Number of words to consider as features
max_length = 100   # Maximum length of each input sequence

In [ ]:
tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(X_train)
X_train_seq = tokenizer.texts_to_sequences(X_train)
X_test_seq = tokenizer.texts_to_sequences(X_test)

X_train_padded = pad_sequences(X_train_seq, maxlen=max_length)
X_test_padded = pad_sequences(X_test_seq, maxlen=max_length)

In [ ]:
# Build the CNN model
model = Sequential()
model.add(Embedding(input_dim=max_words, output_dim=128, input_length=max_length))
model.add(Conv1D(filters=128, kernel_size=5, activation='relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(Flatten())
model.add(Dense(10, activation='relu'))  # Hidden layer
model.add(Dense(2, activation='softmax'))  # Output layer for binary classification

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


In [ ]:
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])


In [ ]:
model.fit(X_train_padded, y_train, epochs=5, batch_size=32, validation_data=(X_test_padded, y_test))


Epoch 1/5
50/50 ━━━━━━━━━━━━━━━━━━━━ 7s 89ms/step - accuracy: 0.5070 - loss: 0.6982 - val_accuracy: 0.4975 - val_loss: 0.6932
Epoch 2/5
50/50 ━━━━━━━━━━━━━━━━━━━━ 5s 98ms/step - accuracy: 0.5458 - loss: 0.6884 - val_accuracy: 0.5050 - val_loss: 0.6915
Epoch 3/5
50/50 ━━━━━━━━━━━━━━━━━━━━ 5s 92ms/step - accuracy: 0.6929 - loss: 0.5917 - val_accuracy: 0.5275 - val_loss: 0.6912
Epoch 4/5
50/50 ━━━━━━━━━━━━━━━━━━━━ 4s 68ms/step - accuracy: 0.9401 - loss: 0.3564 - val_accuracy: 0.5775 - val_loss: 0.6927
Epoch 5/5
50/50 ━━━━━━━━━━━━━━━━━━━━ 5s 70ms/step - accuracy: 0.9923 - loss: 0.3001 - val_accuracy: 0.6125 - val_loss: 0.6798


In [ ]:
# Evaluate the model
loss, accuracy = model.evaluate(X_test_padded, y_test)
print(f"Model accuracy: {accuracy*100:.2f}%")

13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step - accuracy: 0.6076 - loss: 0.6899
Model accuracy: 61.25%
